# SVM 6 emotions

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

Load data and scale it
Remove highly correlated features
Create train and test splits

In [2]:
# data loading
dataset = pd.read_csv('../dataset/labeled_6_combined_dataset.csv', index_col=0)

data = dataset.iloc[:, :-4]
labels = dataset.iloc[:, -1]
unique_labels = np.unique(labels)

# scaling the data
data_scaled = StandardScaler().fit_transform(data)
data_scaled = pd.DataFrame(data_scaled, columns=data.columns)

# removing correlated features
corr = data.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
data.drop(to_drop, axis=1, inplace=True)

# creating train and test splits
data_train, data_test, labels_train, labels_test = train_test_split(data_scaled, labels, test_size=0.1, random_state=0)

# ECG

Extract only ecg related features

In [3]:
ecg_columns = [col for col in data.columns if 'ECG' in col or 'HRV' in col]

ecg_data_train = data_train[ecg_columns]
ecg_data_test = data_test[ecg_columns]

Define the model and train it

In [4]:
ecg_svc = SVC(
    C=1.0,
    kernel='rbf',
    gamma=0.001,
    max_iter=-1,
    decision_function_shape='ovr'
)

In [5]:
ecg_svc.fit(ecg_data_train, labels_train)
ecg_predictions = ecg_svc.predict(ecg_data_test)

In [6]:
ecg_cr = classification_report(labels_test, ecg_predictions, target_names=unique_labels, zero_division=0, digits=4, output_dict=True)
print(classification_report(labels_test, ecg_predictions, target_names=unique_labels, zero_division=0, digits=4))

              precision    recall  f1-score   support

  Ekscytacja     0.0000    0.0000    0.0000       115
   Neutralna     0.2336    0.9919    0.3781       247
  Odprezenie     0.0000    0.0000    0.0000       165
      Radosc     0.3750    0.0187    0.0357       160
      Smutek     0.0000    0.0000    0.0000       151
       Zlosc     0.0000    0.0000    0.0000       219

    accuracy                         0.2346      1057
   macro avg     0.1014    0.1684    0.0690      1057
weighted avg     0.1113    0.2346    0.0938      1057



# EDA

Extract only ecg related features

In [7]:
eda_columns = [col for col in data.columns if 'EDA' in col or 'SCR' in col]

eda_data_train = data_train[eda_columns]
eda_data_test = data_test[eda_columns]

Define the model and train it

In [8]:
eda_svc = SVC(
    C=1.0,
    kernel='rbf',
    gamma=0.001,
    max_iter=-1,
    decision_function_shape='ovr'
)

In [9]:
eda_svc.fit(eda_data_train, labels_train)
eda_predictions = eda_svc.predict(eda_data_test)

In [10]:
eda_cr = classification_report(labels_test, eda_predictions, target_names=unique_labels, zero_division=0, digits=4, output_dict=True)
print(classification_report(labels_test, eda_predictions, target_names=unique_labels, zero_division=0, digits=4))

              precision    recall  f1-score   support

  Ekscytacja     0.0000    0.0000    0.0000       115
   Neutralna     0.2337    1.0000    0.3788       247
  Odprezenie     0.0000    0.0000    0.0000       165
      Radosc     0.0000    0.0000    0.0000       160
      Smutek     0.0000    0.0000    0.0000       151
       Zlosc     0.0000    0.0000    0.0000       219

    accuracy                         0.2337      1057
   macro avg     0.0389    0.1667    0.0631      1057
weighted avg     0.0546    0.2337    0.0885      1057



# Combined

In [11]:
svc = SVC(
    C=1.0,
    kernel='rbf',
    gamma=0.001,
    max_iter=-1,
    decision_function_shape='ovr'
)

In [12]:
svc.fit(data_train, labels_train)
predictions = svc.predict(data_test)

In [13]:
cr = classification_report(labels_test, predictions, target_names=unique_labels, zero_division=0, digits=4, output_dict=True)
print(classification_report(labels_test, predictions, target_names=unique_labels, zero_division=0, digits=4))

              precision    recall  f1-score   support

  Ekscytacja     0.0000    0.0000    0.0000       115
   Neutralna     0.2336    0.9919    0.3781       247
  Odprezenie     0.0000    0.0000    0.0000       165
      Radosc     0.3750    0.0187    0.0357       160
      Smutek     0.0000    0.0000    0.0000       151
       Zlosc     0.0000    0.0000    0.0000       219

    accuracy                         0.2346      1057
   macro avg     0.1014    0.1684    0.0690      1057
weighted avg     0.1113    0.2346    0.0938      1057



# Summary

In [14]:
print(f'''
\tAccuracy:
ECG:\t{ecg_cr['accuracy']}
EDA:\t{eda_cr['accuracy']}
Both:\t{cr['accuracy']}
\tAverage F1:
ECG:\t{ecg_cr['macro avg']['f1-score']}
EDA:\t{eda_cr['macro avg']['f1-score']}
Both:\t{cr['macro avg']['f1-score']}
\tWeighted F1:
ECG:\t{ecg_cr['weighted avg']['f1-score']}
EDA:\t{eda_cr['weighted avg']['f1-score']}
Both:\t{cr['weighted avg']['f1-score']}
''')


	Accuracy:
ECG:	0.2346263008514664
EDA:	0.2336802270577105
Both:	0.2346263008514664
	Average F1:
ECG:	0.06896678424456203
EDA:	0.06313905930470347
Both:	0.06896678424456203
	Weighted F1:
ECG:	0.0937574563796576
EDA:	0.08852609828720014
Both:	0.0937574563796576

